# Exploration des Données Multi-Omiques

Ce notebook présente l'analyse exploratoire des données transcriptomiques et cliniques.

In [ ]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuration pour les visualisations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configuration matplotlib pour les graphiques de meilleure qualité
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Environnement configuré pour l'EDA")

## 1. Chargement des Données

Chargement des données d'expression et cliniques de démonstration.

In [ ]:
# Chemins des fichiers de données
expression_path = '../demo_expression_data.csv'
clinical_path = '../demo_clinical_data.csv'

# Chargement des données
expression_data = pd.read_csv(expression_path)
clinical_data = pd.read_csv(clinical_path)

print("📊 Données chargées avec succès")
print(f"Données d'expression : {expression_data.shape}")
print(f"Données cliniques : {clinical_data.shape}")

## 2. Aperçu des Données

Examen rapide de la structure et du contenu des données.

In [ ]:
# Aperçu des données d'expression
print("=== DONNÉES D'EXPRESSION ===")
print(expression_data.head())
print(f"\nTypes de données : {expression_data.dtypes.to_dict()}")
print(f"\nValeurs manquantes : {expression_data.isnull().sum().sum()}")

# Aperçu des données cliniques
print("\n=== DONNÉES CLINIQUES ===")
print(clinical_data.head())
print(f"\nTypes de données : {clinical_data.dtypes.to_dict()}")
print(f"\nValeurs manquantes : {clinical_data.isnull().sum().sum()}")

## 3. Analyse des Données d'Expression

Distribution des expressions géniques et identification des outliers.

In [ ]:
# Sélectionner les colonnes d'expression (exclure patient_id)
expression_genes = expression_data.drop('patient_id', axis=1)

# Statistiques descriptives
print("=== STATISTIQUES DESCRIPTIVES - EXPRESSION GÉNIQUE ===")
print(expression_genes.describe())

# Vérifier la distribution des valeurs
print("\n=== DISTRIBUTION DES EXPRESSIONS ===")
for gene in expression_genes.columns:
    print(f"{gene}: Min={expression_genes[gene].min():.2f}, Max={expression_genes[gene].max():.2f}, Mean={expression_genes[gene].mean():.2f}")

In [ ]:
# Visualisation : Distribution des expressions géniques
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i, gene in enumerate(expression_genes.columns):
    # Histogramme
    axes[i].hist(expression_genes[gene].dropna(), bins=10, alpha=0.7, color='skyblue', edgecolor='black')
    axes[i].set_title(f'Distribution de {gene}')
    axes[i].set_xlabel('Expression')
    axes[i].set_ylabel('Fréquence')
    
    # Ajouter des statistiques sur le graphique
    mean_val = expression_genes[gene].mean()
    median_val = expression_genes[gene].median()
    axes[i].axvline(mean_val, color='red', linestyle='--', label=f'Moyenne: {mean_val:.1f}')
    axes[i].axvline(median_val, color='green', linestyle='--', label=f'Médiane: {median_val:.1f}')
    axes[i].legend()

plt.tight_layout()
plt.savefig('../results/expression_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Heatmap des corrélations entre gènes
plt.figure(figsize=(10, 8))
correlation_matrix = expression_genes.corr()

# Créer la heatmap
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            cmap='coolwarm', 
            center=0,
            square=True,
            fmt='.3f')

plt.title('Matrice de Corrélation - Expressions Géniques', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Analyse des Données Cliniques

Distribution des variables cliniques et leurs corrélations.

In [ ]:
# Analyse des variables cliniques
print("=== ANALYSE DES VARIABLES CLINIQUES ===")

# Variables catégorielles
categorical_vars = ['sex', 'stage', 'treatment_response']
for var in categorical_vars:
    print(f"\n{var.upper()}:")
    print(clinical_data[var].value_counts())
    print(f"Missing values: {clinical_data[var].isnull().sum()}")

# Variables numériques
numerical_vars = ['age', 'survival_months']
print("\n=== STATISTIQUES NUMÉRIQUES ===")
print(clinical_data[numerical_vars].describe())

In [ ]:
# Visualisation des variables cliniques
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Distribution de l'âge
axes[0,0].hist(clinical_data['age'].dropna(), bins=8, alpha=0.7, color='lightcoral', edgecolor='black')
axes[0,0].set_title('Distribution de l\'âge')
axes[0,0].set_xlabel('Âge (années)')
axes[0,0].set_ylabel('Fréquence')

# Distribution de la survie
axes[0,1].hist(clinical_data['survival_months'], bins=8, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0,1].set_title('Distribution de la survie (mois)')
axes[0,1].set_xlabel('Survie (mois)')
axes[0,1].set_ylabel('Fréquence')

# Distribution par sexe
sex_counts = clinical_data['sex'].value_counts()
axes[1,0].bar(sex_counts.index, sex_counts.values, color=['lightblue', 'pink'], alpha=0.7)
axes[1,0].set_title('Répartition par sexe')
axes[1,0].set_xlabel('Sexe')
axes[1,0].set_ylabel('Nombre de patients')

# Distribution par stade
stage_counts = clinical_data['stage'].value_counts()
axes[1,1].bar(stage_counts.index, stage_counts.values, color='orange', alpha=0.7)
axes[1,1].set_title('Répartition par stade')
axes[1,1].set_xlabel('Stade')
axes[1,1].set_ylabel('Nombre de patients')

plt.tight_layout()
plt.savefig('../results/clinical_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Intégration des Données

Fusion des données d'expression et cliniques pour l'analyse intégrative.

In [ ]:
# Fusion des données sur patient_id
integrated_data = pd.merge(expression_data, clinical_data, on='patient_id', how='inner')

print("=== DONNÉES INTÉGRÉES ===")
print(f"Dimensions après fusion : {integrated_data.shape}")
print(f"Valeurs manquantes totales : {integrated_data.isnull().sum().sum()}")

# Aperçu des données intégrées
print("\nAperçu des données intégrées :")
print(integrated_data.head())

# Sauvegarder les données intégrées
output_path = '../results/integrated_demo_data.csv'
Path('../results').mkdir(exist_ok=True)
integrated_data.to_csv(output_path, index=False)
print(f"\n✅ Données intégrées sauvegardées : {output_path}")

## 6. Analyse de Qualité

Évaluation de la qualité des données et identification des problèmes.

In [ ]:
# Rapport de qualité des données
def generate_quality_report(data, data_name):
    """Génère un rapport de qualité pour un jeu de données"""
    report = {
        'dataset': data_name,
        'shape': data.shape,
        'missing_values': data.isnull().sum().to_dict(),
        'data_types': data.dtypes.to_dict(),
        'duplicates': data.duplicated().sum(),
        'completeness': 1 - (data.isnull().sum().sum() / (data.shape[0] * data.shape[1]))
    }
    return report

# Générer les rapports
expression_report = generate_quality_report(expression_data, 'Expression')
clinical_report = generate_quality_report(clinical_data, 'Clinical')
integrated_report = generate_quality_report(integrated_data, 'Integrated')

print("=== RAPPORTS DE QUALITÉ ===\n")

for report in [expression_report, clinical_report, integrated_report]:
    print(f"Dataset: {report['dataset']}")
    print(f"  Dimensions: {report['shape']}")
    print(f"  Complétude: {report['completeness']:.3f}")
    print(f"  Duplicatas: {report['duplicates']}")
    print(f"  Valeurs manquantes: {sum(report['missing_values'].values())}")
    print("-" * 40)
    print()

## 7. Conclusions et Recommandations

Synthèse de l'analyse et identification des prochaines étapes.

In [ ]:
# Synthèse de l'analyse
print("=== SYNTHÈSE DE L'ANALYSE EXPLORATOIRE ===\n")

print("1. QUALITÉ DES DONNÉES:")
print(f"   - Données d'expression: {expression_data.shape[0]} échantillons, {expression_data.shape[1]-1} gènes")
print(f"   - Données cliniques: {clinical_data.shape[0]} échantillons, {clinical_data.shape[1]-1} variables")
print(f"   - Complétude globale: {integrated_report['completeness']:.1%}")

print("2. CARACTÉRISTIQUES DES DONNÉES:")
print(f"   - Âge moyen: {clinical_data['age'].mean():.1f} ± {clinical_data['age'].std():.1f} ans")
print(f"   - Survie médiane: {clinical_data['survival_months'].median():.0f} mois")
print(f"   - Distribution des stades: {dict(clinical_data['stage'].value_counts())}")

print("3. PROBLÈMES IDENTIFIÉS:")
missing_total = integrated_data.isnull().sum().sum()
if missing_total > 0:
    print(f"   - Valeurs manquantes: {missing_total} valeurs ({missing_total/(integrated_data.shape[0]*integrated_data.shape[1]):.1%})")
else:
    print("   - Aucune valeur manquante détectée")

print("4. RECOMMANDATIONS POUR LE PRÉTRAITEMENT:")
print("   - Imputation des valeurs manquantes (KNN ou médiane)")
print("   - Normalisation des données d'expression (TMM ou DESeq2)")
print("   - Vérification des outliers dans les données d'expression")
print("   - Validation de la qualité des échantillons")

print("✅ Analyse exploratoire terminée avec succès!")
print("📁 Résultats sauvegardés dans /results/")